## Programming Drill 3.2.2
### What would happen if there were more than two slits? Write a program that asks a user to design a multislit experiment. The user notes the number of slits and the number of targets to measure the bullets. Then the user enters probabilities of the bullets' moving from each slit to each target. An appropriate matrix is setup and then the matrix is multiplied by itself. Have the program print the appropriate resulting matrix and vector.


In [2]:
#!/usr/bin/env python3
"""
Generalized multislit B-matrix builder (Programming Drill 3.2.2)

Vertices:
  0         : gun
  1..s      : slits
  s+1..s+m  : targets (absorbing)

Matrix B is (1+s+m) x (1+s+m), rows = next-position, cols = current-position.
"""

from fractions import Fraction
from typing import List

def F(x: str) -> Fraction:
    """Parse '1/2', '0.25', '1' etc. into Fraction."""
    x = x.strip()
    try:
        return Fraction(x)
    except Exception:
        try:
            return Fraction(float(x))
        except Exception:
            raise ValueError(f"Cannot parse '{x}' as a rational number.")

def read_int(prompt: str) -> int:
    while True:
        try:
            v = int(input(prompt).strip())
            if v <= 0:
                print("Enter a positive integer.")
                continue
            return v
        except Exception:
            print("Enter a valid integer.")

def read_prob_vector(n: int, prompt: str) -> List[Fraction]:
    while True:
        line = input(prompt).strip()
        parts = line.split()
        if len(parts) != n:
            print(f"Expected {n} entries, got {len(parts)}.")
            continue
        try:
            vec = [F(p) for p in parts]
        except Exception as e:
            print(e)
            continue
        if any(x < 0 for x in vec):
            print("Probabilities must be non-negative.")
            continue
        s = sum(vec)
        if s != Fraction(1):
            print(f"Probabilities must sum to 1 exactly; they sum to {s}.")
            continue
        return vec

def make_B(s: int, m: int,
           gun_to_slits: List[Fraction],
           slit_to_targets: List[List[Fraction]]) -> List[List[Fraction]]:
    n = 1 + s + m
    B = [[Fraction(0) for _ in range(n)] for __ in range(n)]

    # gun column (col 0) -> slits 1..s
    for i in range(s):
        B[1 + i][0] = gun_to_slits[i]

    # each slit column j (col = 1+j) -> targets rows s+1 .. s+m
    for j in range(s):
        col = 1 + j
        for t in range(m):
            row = 1 + s + t
            B[row][col] = slit_to_targets[j][t]

    # targets absorb: for each target vertex p = s+1..s+m, B[p][p] = 1
    for t in range(m):
        idx = 1 + s + t
        B[idx][idx] = Fraction(1)

    return B

def mat_mult(A: List[List[Fraction]], B: List[List[Fraction]]) -> List[List[Fraction]]:
    p = len(A)
    q = len(A[0]) if p>0 else 0
    assert q == len(B), "Incompatible dims for multiplication"
    r = len(B[0])
    C = [[Fraction(0) for _ in range(r)] for __ in range(p)]
    for i in range(p):
        for j in range(r):
            s = Fraction(0)
            for k in range(q):
                s += A[i][k] * B[k][j]
            C[i][j] = s
    return C

def mat_vec(A: List[List[Fraction]], v: List[Fraction]) -> List[Fraction]:
    m = len(A)
    n = len(A[0]) if m>0 else 0
    assert n == len(v), "Incompatible dims mat-vec"
    res = [Fraction(0) for _ in range(m)]
    for i in range(m):
        s = Fraction(0)
        for j in range(n):
            s += A[i][j] * v[j]
        res[i] = s
    return res

def print_matrix(A: List[List[Fraction]], name: str = "Matrix"):
    print(f"\n{name} ({len(A)} x {len(A[0])}):")
    for row in A:
        print("  " + "  ".join(str(x) for x in row))
    print()

def print_vector(v: List[Fraction], name: str = "Vector"):
    print(f"\n{name}:")
    print("  " + "  ".join(str(x) for x in v))
    print()

def main():
    print("Multislit generalized example — Programming Drill 3.2.2\n")
    s = read_int("Enter number of slits (s): ")
    m = read_int("Enter number of targets (m): ")

    print(f"\nEnter gun -> slits probabilities ({s} numbers summing to 1).")
    gun_to_slits = read_prob_vector(s, f"gun->slits (space-separated {s} probs): ")

    slit_to_targets: List[List[Fraction]] = []
    for j in range(s):
        print(f"\nEnter probabilities slit {j+1} -> each of the {m} targets (must sum to 1).")
        vec = read_prob_vector(m, f"slit {j+1} -> targets (space-separated {m} probs): ")
        slit_to_targets.append(vec)

    # build B
    B = make_B(s, m, gun_to_slits, slit_to_targets)
    print_matrix(B, "B (rows=next, cols=current)")

    # initial vector: bullet at gun
    n = 1 + s + m
    v0 = [Fraction(0) for _ in range(n)]
    v0[0] = Fraction(1)
    print_vector(v0, "v0 (initial: bullet at gun)")

    # one click
    v1 = mat_vec(B, v0)
    print_vector(v1, "v1 = B * v0  (after 1 click)")

    # B^2 and v2
    B2 = mat_mult(B, B)
    print_matrix(B2, "B^2")

    v2 = mat_vec(B2, v0)
    print_vector(v2, "v2 = B^2 * v0  (after 2 clicks)")

if __name__ == "__main__":
    main()


Multislit generalized example — Programming Drill 3.2.2


Enter gun -> slits probabilities (3 numbers summing to 1).

Enter probabilities slit 1 -> each of the 5 targets (must sum to 1).

Enter probabilities slit 2 -> each of the 5 targets (must sum to 1).

Enter probabilities slit 3 -> each of the 5 targets (must sum to 1).

B (rows=next, cols=current) (9 x 9):
  0  0  0  0  0  0  0  0  0
  1/3  0  0  0  0  0  0  0  0
  1/3  0  0  0  0  0  0  0  0
  1/3  0  0  0  0  0  0  0  0
  0  1/5  1/5  1/5  1  0  0  0  0
  0  1/5  1/5  1/5  0  1  0  0  0
  0  1/5  1/5  1/5  0  0  1  0  0
  0  1/5  1/5  1/5  0  0  0  1  0
  0  1/5  1/5  1/5  0  0  0  0  1


v0 (initial: bullet at gun):
  1  0  0  0  0  0  0  0  0


v1 = B * v0  (after 1 click):
  0  1/3  1/3  1/3  0  0  0  0  0


B^2 (9 x 9):
  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0
  1/5  1/5  1/5  1/5  1  0  0  0  0
  1/5  1/5  1/5  1/5  0  1  0  0  0
  1/5  1/5  1/5  1/5  0